In [10]:
import os
import pdfplumber
import re

Ségmentation par mot clés et blocs

In [11]:
def segmenter_texte_par_mots_cles(texte):
    sections = {
        "Experience": [],
        "Formation": [],
        "Competences": [],
        "Autre": []
    }

    keywords = {
        "Experience": ["experience", "expériences", "parcours", "postes", "emploi", "expériences professionnelles"],
        "Formation": ["formation", "education", "diplomes", "études", "cursus"],
        "Competences": ["competences", "hard skills", "outils", "langues", "logiciels", "atouts", "compétences techniques"]
    }

    current_section = "Autre"

    lignes = texte.split('\n')

    for ligne in lignes:
        ligne_clean = ligne.strip()
        if not ligne_clean:
            continue

        ligne_lower = ligne_clean.lower()

        if len(ligne_clean) < 40:
            for section, triggers in keywords.items():
                if any(trigger in ligne_lower for trigger in triggers):
                    current_section = section
                    break

        sections[current_section].append(ligne_clean)

    return sections


Lecture en deux blocs et stucturation

In [12]:
def extraire_cv_colonnes(pdf_path, ratio_colonne=0.35):

    contenu_final = {
        "Experience": [],
        "Formation": [],
        "Competences": [],
        "Autre": []
    }

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                width = page.width
                height = page.height

                # ✅ Sécurisation de la coupure de colonnes
                x_split = max(1, min(width * ratio_colonne, width - 1))

                bbox_gauche = (0, 0, x_split, height)
                zone_gauche = page.crop(bbox_gauche)
                texte_gauche = zone_gauche.extract_text() or ""

                bbox_droite = (x_split, 0, width, height)
                zone_droite = page.crop(bbox_droite)
                texte_droite = zone_droite.extract_text() or ""

                dict_gauche = segmenter_texte_par_mots_cles(texte_gauche)
                dict_droite = segmenter_texte_par_mots_cles(texte_droite)

                for key in contenu_final:
                    contenu_final[key].extend(dict_gauche[key])
                    contenu_final[key].extend(dict_droite[key])

        return {k: " ".join(v) for k, v in contenu_final.items()}

    except Exception as e:
        print(f"Erreur avec pdfplumber sur {pdf_path}: {e}")
        return None


Traitement automatique de tout les CVs

In [13]:
dossier_cv = "CV pas bien"   # tous les CV bruts

liste_fichiers = [f for f in os.listdir(dossier_cv) if f.endswith(".pdf")]


Boucle automatique sur tout les CV

In [14]:
for fichier in liste_fichiers:
    chemin_cv = os.path.join(dossier_cv, fichier)

    resultat = extraire_cv_colonnes(chemin_cv, ratio_colonne=0.30)

    if resultat:
        print("\n" + "="*80)
        print(f"📄 ANALYSE DE : {fichier}")
        print("="*80)

        print("\n🟦 COMPÉTENCES :")
        print(resultat["Competences"][:500] + "...\n")

        print("🟨 EXPÉRIENCE :")
        print(resultat["Experience"][:500] + "...\n")

        print("🟩 FORMATION :")
        print(resultat["Formation"][:500] + "...\n")


Erreur avec pdfplumber sur CV pas bien\CVCamillenonano.pdf: Bounding box (0, 0, 178.65, 842.2499787) is not fully within parent page bounding box (0.0, -7.829981299999986, 595.5, 834.4199974)

📄 ANALYSE DE : CVchimisteanglais.pdf

🟦 COMPÉTENCES :
...

🟨 EXPÉRIENCE :
• 1 year of experience in independently • 2 months as an organic ch...

🟩 FORMATION :
EDUCATION Master of Science in Chemistr • Master thesis: “Towards • Master project: “Lantha reaction”. Grade: 6/6. Bachelor of Science in Chemis High School Diploma, Lycée In • Degree awarded with hig PROFESSIONAL EX Bioinorganic Chemistry Intern University of Geneva, Prof. Milto • Lead an innovative pro promising leads and now • Purified proteins under • Measured the activity of • Delivered oral and writt Organic Chemistry Intern University of Geneva, Prof. Pigue • Participated in a projec lead...


📄 ANALYSE DE : CVchimistenonano.pdf

🟦 COMPÉTENCES :
...

🟨 EXPÉRIENCE :
...

🟩 FORMATION :
EDUCATION Master en Chimie, Université d • Thèse 